In [8]:
# !pip install pyautogui comtypes pycaw

In [9]:

import os
import numpy as np
import cv2
import mediapipe as mp
import pyautogui
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [10]:
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
vol_range = volume.GetVolumeRange()
min_vol, max_vol = vol_range[0], vol_range[1]

In [11]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=2,
                       min_detection_confidence=0.7,
                       min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(1)
screen_w, screen_h = pyautogui.size()


In [12]:
def get_landmark_coords(hand_landmarks, frame):
    height, width, _ = frame.shape
    coords = []
    for lm in hand_landmarks.landmark:
        cx, cy = int(lm.x * width), int(lm.y * height)
        coords.append((cx, cy))
    return coords


In [13]:

import time
last_open_time = 0
cooldown = 5  # seconds


In [14]:
def detect_hand_type(handedness):
    return handedness.classification[0].label  # 'Left' or 'Right'

while True:
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            h, w, _ = frame.shape
            hand_type = detect_hand_type(handedness)

            # Get all landmarks
            landmarks = [(int(lm.x * w), int(lm.y * h)) for lm in hand_landmarks.landmark]

            # Draw landmarks
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            if hand_type == 'Right':
                # === Volume Control ===
                x1, y1 = landmarks[4]   # Thumb tip
                x2, y2 = landmarks[8]   # Index tip

                length = np.hypot(x2 - x1, y2 - y1)
                vol = np.interp(length, [30, 200], [min_vol, max_vol])
                volume.SetMasterVolumeLevel(vol, None)

                vol_percent = np.interp(length, [30, 200], [0, 100])
                vol_bar_height = np.interp(length, [30, 200], [400, 150])  # Adjust bar height

                if vol_percent < 30:
                    vol_bar_color =(0,0,255)
                
                elif vol_percent < 80:
                    vol_bar_color = (0, 255, 255)
                
                else:
                    vol_bar_color = (0, 255, 0)

                # Draw volume bar
                cv2.rectangle(frame, (50, 150), (85, 400), (255, 255, 255), 2)
                cv2.rectangle(frame, (50, int(vol_bar_height)), (85, 400), vol_bar_color, -1)

                # Draw volume percentage
                cv2.putText(frame, f'{int(vol_percent)} %', (40, 430),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, vol_bar_color, 2)


                # === Cursor Control ===
                cursor_x, cursor_y = landmarks[8]  # Index finger tip
                screen_x = np.interp(cursor_x, [0, w], [0, screen_w])
                screen_y = np.interp(cursor_y, [0, h], [0, screen_h])
                pyautogui.moveTo(screen_x, screen_y, duration=0.01)

                cv2.putText(frame, 'Right Hand: Volume + Mouse', (10, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
                

            elif hand_type == 'Left':
                gesture_found = False 

                if landmarks[8][1] < landmarks[6][1] and landmarks[12][1] < landmarks[10][1]:
                    cv2.putText(frame, 'Peace', (10, 80),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
                    gesture_found = True
                    

                elif landmarks[4][1] < landmarks[3][1] and all(landmarks[i][1] > landmarks[i - 2][1] for i in [8, 12, 16, 20]):
                    cv2.putText(frame, 'Thumbs Up', (10, 100),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    gesture_found = True

                elif landmarks[8][1] < landmarks[6][1] and landmarks[20][1] < landmarks[18][1] \
                and landmarks[12][1] > landmarks[10][1] and landmarks[16][1] > landmarks[14][1]:
                    cv2.putText(frame, 'Rock', (10, 120),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (128, 0, 128), 2)
                    gesture_found = True

                elif all(landmarks[i][1] < landmarks[i - 2][1] for i in [8, 12, 16, 20]) and landmarks[4][0] < landmarks[3][0]:
                    cv2.putText(frame, 'Paper', (10, 140),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 128, 255), 2)
                    gesture_found = True

                elif np.hypot(landmarks[4][0] - landmarks[8][0], landmarks[4][1] - landmarks[8][1]) < 20 and \
                    all(landmarks[i][1] < landmarks[i - 2][1] for i in [12, 16, 20]):
                    cv2.putText(frame, 'OK Sign - Explorer', (10, 180),
                                 cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 128, 255), 2)
                    gesture_found = True
                    if time.time() - last_open_time > cooldown:
                        os.system("explorer")
                        last_open_time = time.time()
                    
                elif landmarks[4][1] < landmarks[3][1] and landmarks[20][1] < landmarks[18][1] \
                and all(landmarks[i][1] > landmarks[i - 2][1] for i in [8, 12, 16]):
                    cv2.putText(frame, 'Yo', (10, 160),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    gesture_found = True

                if not gesture_found:
                    cv2.putText(frame, 'Unidentified', (10, 160),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


                    # For clicking action, put thumb and index finger close to each other
                    x1, y1 = landmarks[4]   # For Thumb
                    x2, y2 = landmarks[8]   # For Index
                    click_dist = np.hypot(x2 - x1, y2 - y1)
                    if click_dist < 20:
                        pyautogui.click()
                        cv2.putText(frame, 'Click!', (10, 110),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Dual-Hand Control", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()



Exception ignored in: <function _compointer_base.__del__ at 0x000002452CDECB80>
Traceback (most recent call last):
  File "c:\Users\Nirajan\Desktop\Internship_Projects\Hand_Gesture\env\Lib\site-packages\comtypes\_post_coinit\unknwn.py", line 284, in __del__
    self.Release()  # type: ignore
    ^^^^^^^^^^^^^^
  File "c:\Users\Nirajan\Desktop\Internship_Projects\Hand_Gesture\env\Lib\site-packages\comtypes\_post_coinit\unknwn.py", line 418, in Release
    return self.__com_Release()  # type: ignore
           ^^^^^^^^^^^^^^^^^^^^
OSError: exception: access violation writing 0x0000000000000000
